## Lecture-4: Computational Graph Calculation


01. Simple Back Propagation

![Back-1:](../images/01_BackPropagation.png)

In [36]:
'''
f(x,y,z)=(x+y)Z ex: x=-2, y= 5, z=-4
q=x+y; dq/dx=1; dq/dy=1
f=qz; df/dq=z; df/dz=q
'''
class SimplePropagation:
    def __init__(self,x,y,z):
        self.x=x
        self.y=y
        self.z=z
    def forward(self):
        self.q= self.x+self.y # q=p+q
        self.f= self.q*self.z # z=qz
    def back(self): #dout = df/df=1
        self.dq=self.z # df/dq = (df/dz)=1*z
        self.dz=self.q # df/dz = (df/dz)= 1*q
        self.dx=self.dq # df/dx=(df/dq)*(dq/dx)=z*1
        self.dy=self.dq # df/dy=(df/dq)*(dq/dy)=z*1

s=SimplePropagation(-2,5,-4)
s.forward()
s.back()
print(f"forward: q={s.q} f={s.f}")
print(f"back: dq={s.dq} dz={s.dz} dx={s.dx} dy:{s.dy}")

forward: q=3 f=-12
back: dq=-4 dz=3 dx=-4 dy:-4


2. Complex Calculation

![Image-1:](../images/02_backPropagation.png)

In [12]:
import numpy as np
'''
f(w,x) =1/1+e^-(W0X0+W1X1+W2X2)
p=W0*X0
q=W1*X1
r=p+q
s=r+w2
s1=s*(-1)
s2=exp(s1)
s3=s2+1
f=1/s3
'''
class Propagation:
    def __init__(self,X0,W0,X1,W1,W2):
        self.X0=X0
        self.W0=W0
        self.X1=X1
        self.W1=W1
        self.W2=W2
    def forward(self):
        self.p= self.X0*self.W0 # 
        self.q= self.X1*self.W1 # 
        self.r=self.p+self.q
        self.s= self.r+self.W2
        self.s1=self.s*(-1)
        self.s2= np.exp(self.s1)
        self.s3=self.s2+1
        self.f=1/self.s3
    def back(self): #dout = df/df=1
        self.df= -1/self.s3**2 # df/dx = d(1/x)/dx=-1/x^2
        self.ds3= 1*self.df # dcf(x)/dx = c+x
        self.ds2= np.exp(-1)*self.ds3 #f(x)/dx= f(exe^x)/dx= exe^x
        self.ds1= (-1)* self.ds2 #f(x)/dx= f(exe^x)/dx= exe^x
        self.dw2=(1)*self.ds1 # df/dw2 = f(r+w2)/dw2= 1
        self.dr=(1)*self.ds1 # df/dr = f(r+w2)/dr= 1
        self.dp=(1)*self.dr # df/dp = f(p+q)/dp= 1
        self.dq=(1)*self.dr # df/dq = f(p+q)/dq= 1
        self.dw1=self.dq*self.X1 # df/dw1 = f(q)/dw1=f(w1x1)/dw1= 1*x1
        self.dx1=self.dq*self.W1 # df/dx1 = f(q)/dx1=f(w1x1)/dx1= 1*w1
        self.dw0=self.dp*self.X0 # df/dw0 = f(p)/dw0=f(w0x0)/dw0= 1*x0
        self.dx0=self.dp*self.W0 # df/dx1 = f(p)/dx0=f(w0x0)/dx0= 1*w0

s= Propagation(-1,2,-2,-3,-3) #
s.forward()
s.back()
print(f"Forward Propagation:\n p=W0*X0: {s.p} q=W1*X1:{s.q} r=p+q:{s.r} s=r+w2:{s.s} s1=s*(-1):{s.s1} s2=exp(s1):{s.s2} s3=s2+1:{s.s3} f=1/s3:{s.f}")
print("Back Propagation:")
print(f"d(f)/df:{s.df} d(1/x)/dx:{s.ds3} dcf(x)/dx: {s.ds2} f(exe^x)/dx: {s.ds1} ")
print(f"dr: {s.dr} dw2: {s.dw2} dp: {s.dp} dq: {s.dq}")
print(f"dw1: {s.dw1} dx1: {s.dx1} dw0: {s.dw0} dx0: {s.dx0}")

Forward Propagation:
 p=W0*X0: -2 q=W1*X1:6 r=p+q:4 s=r+w2:1 s1=s*(-1):-1 s2=exp(s1):0.36787944117144233 s3=s2+1:1.3678794411714423 f=1/s3:0.7310585786300049
Back Propagation:
d(f)/df:-0.534446645388523 d(1/x)/dx:-0.534446645388523 dcf(x)/dx: -0.19661193324148188 f(exe^x)/dx: 0.19661193324148188 
dr: 0.19661193324148188 dw2: 0.19661193324148188 dp: 0.19661193324148188 dq: 0.19661193324148188
dw1: -0.39322386648296376 dx1: -0.5898357997244457 dw0: -0.19661193324148188 dx0: 0.39322386648296376


### A vectorized Example
![Image-1:](../images/jacobian-matrix.png)
![Image-2:](../images/03_BackPropagationVectorized.png)

In [30]:
import numpy as np
w=np.array([[.1,.5],[-.3, .8]])
x= np.array([[.2],[.4]])
print(f"Dimention of x={x.shape} and Dimention of w={w.shape}")

def forward(X,w):
    # step-1: dot multiplication
    q=w.dot(X) # q=wx
    out=np.sum(np.square(q)) # f(q)=||q||^2
    return q,out
def backward(q,x,w):
    dgrad= 2*q # df/dqi= 2qi vectorized form or df/dq=2q
    # dw=dgrad*(x)
    xT= np.transpose(x)
    wT=np.transpose(w)
    # dw=np.matmul(dgrad,xT) # df/dw=2q*x^T
    dw=dgrad.dot(xT)
    # dx=np.matmul(wT,dgrad) ## df/dx=w^T*2q
    dx=wT.dot(dgrad)
    return dgrad, dw, dx

q, out= forward(x,w)
print(f"q:{q} out:{out}")
# back
dgrad, dw, dx= backward(q, x, w)
print("dgrad:\n", dgrad)
print("dw:\n", dw)
print("dx:\n", dx)

Dimention of x=(2, 1) and Dimention of w=(2, 2)
q:[[0.22]
 [0.26]] out:0.11600000000000003
dgrad:
 [[0.44]
 [0.52]]
dw:
 [[0.088 0.176]
 [0.104 0.208]]
dx:
 [[-0.112]
 [ 0.636]]


### Forward and Backward Propagation 

In [41]:
import math
w = [2,-3,-3] # assume some random weights and data
x = [-1, -2]

# forward pass
dot = w[0]*x[0] + w[1]*x[1] + w[2]
f = 1.0 / (1 + math.exp(-dot)) # sigmoid function

# backward pass through the neuron (backpropagation)
ddot = (1 - f) * f # gradient on dot variable, using the sigmoid gradient derivation
dx = [w[0] * ddot, w[1] * ddot] # backprop into x
dw = [x[0] * ddot, x[1] * ddot, 1.0 * ddot] # backprop into w
# we're done! we have the gradients on the inputs to the circuit
print(dx)
print(dw)

[0.3932238664829637, -0.5898357997244456]
[-0.19661193324148185, -0.3932238664829637, 0.19661193324148185]


### Staged computation
- $f(x,y)=(x+p(y))/(p(x)+(x+y)^2)$

In [42]:
x = 3 # example values
y = -4

# forward pass
sigy = 1.0 / (1 + math.exp(-y)) # sigmoid in numerator   #(1)
num = x + sigy # numerator                               #(2)
sigx = 1.0 / (1 + math.exp(-x)) # sigmoid in denominator #(3)
xpy = x + y                                              #(4)
xpysqr = xpy**2                                          #(5)
den = sigx + xpysqr # denominator                        #(6)
invden = 1.0 / den                                       #(7)
f = num * invden # done!
print(f)

1.5456448841066441


Computing the backprop pass is easy: We’ll go backwards and for every variable along the way in the forward pass (sigy, num, sigx, xpy, xpysqr, den, invden) we will have the same variable, but one that begins with a d, which will hold the gradient of the output of the circuit with respect to that variable.

In [43]:
# backprop f = num * invden
dnum = invden # gradient on numerator                             #(8)
dinvden = num                                                     #(8)
# backprop invden = 1.0 / den 
dden = (-1.0 / (den**2)) * dinvden                                #(7)
# backprop den = sigx + xpysqr
dsigx = (1) * dden                                                #(6)
dxpysqr = (1) * dden                                              #(6)
# backprop xpysqr = xpy**2
dxpy = (2 * xpy) * dxpysqr                                        #(5)
# backprop xpy = x + y
dx = (1) * dxpy                                                   #(4)
dy = (1) * dxpy                                                   #(4)
# backprop sigx = 1.0 / (1 + math.exp(-x))
dx += ((1 - sigx) * sigx) * dsigx # Notice += !! See notes below  #(3)
# backprop num = x + sigy
dx += (1) * dnum                                                  #(2)
dsigy = (1) * dnum                                                #(2)
# backprop sigy = 1.0 / (1 + math.exp(-y))
dy += ((1 - sigy) * sigy) * dsigy       
print(dy)                          #(1)
# done! phew

1.5922327514838093


#### Modularized implementation: forward / backward API

In [45]:
class MultiplyGate:
    def forward(self,x,y):
        self.x = x
        self.y = y
        z = x*y
        return z
    def backward(dz):
        dx = y * dz # [dz/dz * dL/dz]
        dy = x * dz # [dz/dy * dL/dz]
        return [dx,dy]

## Example feed-forward computation of a neural network

In [35]:
class Neuron:
    def neuron_trics(self, inputs):
        """Assume insputs are 1-D numpy array and bias is a number"""
        cell_body_sum= np.sum(inputs*self.weights)+ self.bias
        firing_rate=1/1+math.exp(-cell_body_sum) # activation function(sigmoid function)
        return firing_rate